# Menschlicher Spieler

Diese Notebook implementiert einen menschlichen Spieler welcher für jeden Zug gefragt wird.  

In [1]:
import random
# Set a seed, so randomization is reproducable
random.seed(3)

In [ ]:
import chess

class HumanPlayer(Exercise01AI):

    def __init__(self, player_name) -> None:
        super().__init__(player_name)

    def make_turn(self: ChessAI, board: chess.Board) -> None:
        """Asks the human to make a turn on the given board."""
        next_move = None

        while True:
            next_move_str = input("Enter your next move as UCI string: ")
            if not next_move_str:
                return self.let_ai_play(board)
            try:
                next_move = chess.Move.from_uci(next_move_str)
            except ValueError:
                print("Illegal input!")
                print("Please use UCI format.")
                continue
            if next_move not in board.legal_moves:
                print("This move is not possible!")
                print("Available moves: ", [move.uci() for move in board.legal_moves])
                continue
            break

        # Push next move
        board.push(next_move)

    def let_ai_play(self: ChessAI, board: chess.Board) -> None:
        """Uses Exercise01AI to make the turn."""
        next_move, next_state = self.get_next_move(board)
        # Push next move or finish game
        if next_move:
            board.push(next_move)
        else:
            print(f"Finished with: {self.state} -> {State.FINISHED}")
            self.state = State.FINISHED
            # Close endgame tablebase reader
            self.end_game_reader.close()
            return

        # Handle state transitions
        if next_state != self.state:
            # Close ployglot opening book to save memory if opening game has finished 
            if self.state == State.OPENING_GAME:
                self.opening_game_reader.close()
            self.state = next_state
